Downloading required packages for OCR and Summarizing

In [ ]:
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.86.1-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
pip install PyPDF2 opencv-python pytesseract Pillow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
import PyPDF2
import cv2
import pytesseract
from PIL import Image
import numpy as np
from pdf2image import convert_from_path

Importing a sample handwritten pdf and storing it

In [ ]:
from google.colab import files
uploaded = files.upload()

We are using the pretrained t5 model, however to get better results we will have to see if there is a better model suited for our purpose (maybe train one ourselves if we get an appropriate training dataset)

In [ ]:
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


The extract text function (Open to change)

In [ ]:
def extract_text_from_image(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

    # Apply any preprocessing steps (e.g., denoising, thresholding, resizing) using OpenCV if needed

    # Noise reduction
    denoised_image = cv2.medianBlur(gray_image, 3)

    # Skew correction
    """
    coords = np.column_stack(np.where(denoised_image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
      angle = -angle
    rows, cols = denoised_image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    corrected_image = cv2.warpAffine(denoised_image, M, (cols, rows), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Thin line enhancement
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2, 2))
    thinned_image = cv2.erode(denoised_image, kernel, iterations=1)
    equalized_image = cv2.equalizeHist(thinned_image)

    """
    """
    cv2_imshow(image)
    cv2_imshow(equalized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    """
    extracted_text = pytesseract.image_to_string(denoised_image)
    print(type(extracted_text))
    return extracted_text



Summarizes the text passed(parameters and algorithms subject to change)

In [ ]:
def generate_summary(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs, num_beams=6,max_length=500, early_stopping=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
pdf_file = "/content/1.pdf"

# Read the PDF document
summary=""
with open(pdf_file, "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)

    total_extract = ""

    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]

        # Convert PDF page to image
        images = convert_from_path(pdf_file, first_page=page_num + 1, last_page=page_num + 1)
        image = images[0]

        # Perform OCR on the image
        extracted_text = extract_text_from_image(image)
        summary += generate_summary(extracted_text)
        total_extract += extracted_text






<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
print(total_extract)

22 / Who Moved My Cheese? ~~

Carlos said, “I guess we resist
because we're afraid of change.”

“Carlos, you were Captain of the footbal
Jessica said. “I never thought I'd hear y
anything about being afraid!”

They all laughed as they realized that although
they had gone off in different directions—fro,,
working at home to managing companies—they
were experiencing similar feelings.

Everyone was trying to cope with the
unexpected changes that were happening to them
in recent years. And most admitted that they did
not know a good way to handle them.

Then Michael said, “I used to be afraid of
change. When a big change came along in our
business, we didn’t know what to do. So we didn't
adjust and we almost lost it.

“That is,” he continued, “until I heard a funny
little story that changed everything.”

“How so?” Nathan asked.

“Well, the story changed the way | looked at
change—from losing something to gaining some
thing—and it showed me how to do it. After that,
things quickly improved

In [ ]:
print("Summary")
print(summary)

Summary
everyone was trying to cope with the unexpected changes that were happening to them in recent years. most admitted that they did not know a good way to handle them. "i was really annoyed with myself for not seeing the obvious and doing what works when things change.""i realized the four characters in the story resented the various parts of myself," he says. many people said it helped them in their personal lives. but a few people said they got nothing out of it.
